In [1]:
import sys
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# 176. Second Highest Salary

如果要return null而不是空值,可以嵌套个字查询

### Method1：

In [6]:
def find_SecondHighestSalary(df):
    salary = list(df.salary.unique()) # get disinct salary
    salary.sort(reverse=True)
    
    if len(salary) < 2: #  If there is no second highest salary, the query should report null
        return pd.DataFrame({'SecondHighestSalary':  [None]})
    
    return pd.DataFrame({'SecondHighestSalary':  [salary[1]]})

df1 = pd.DataFrame({'id':[1,2,3,4,5,6], 'salary': [100,50,300,400,450,450]}) # 如果有max salary不唯一，应该取第二大的salary
df2 = pd.DataFrame({'id':[1,2,3,4,5], 'salary': [100,50,300,400,450]})
df3 = pd.DataFrame({'id':[1], 'salary': [100]})
df4 = pd.DataFrame({'id':[], 'salary': []})

print(find_SecondHighestSalary(df1))
print(find_SecondHighestSalary(df2))
print(find_SecondHighestSalary(df3))
print(find_SecondHighestSalary(df4))

   SecondHighestSalary
0                  400
   SecondHighestSalary
0                  400
  SecondHighestSalary
0                None
  SecondHighestSalary
0                None


### Method2：

In [7]:
def find_SecondHighestSalary2(df):
    if len(df)<2:
        return pd.DataFrame({'SecondHighestSalary':  [None]})
    df = df[df.salary < max(df.salary)] 
    df = df.sort_values('salary', ascending=False).reset_index(drop=True)
    return df[['salary']].iloc[[0]].rename(columns={'salary':'SecondHighestSalary'})

print(find_SecondHighestSalary2(df1))
print(find_SecondHighestSalary2(df2))
print(find_SecondHighestSalary2(df3))
print(find_SecondHighestSalary2(df4))

   SecondHighestSalary
0                  400
   SecondHighestSalary
0                  400
  SecondHighestSalary
0                None
  SecondHighestSalary
0                None


# 595. Big Countries

In [4]:
df = pd.read_clipboard()

In [5]:
df

,name,continent,area,population,gdp
0,Afghanistan,Asia,652230,25500100,20343000000
1,Albania,Europe,28748,2831741,12960000000
2,Algeria,Africa,2381741,37100000,188681000000
3,Andorra,Europe,468,78115,3712000000
4,Angola,Africa,1246700,20609294,100990000000


In [48]:
def big_countries(df):
    return df[(df.area >= 3000000) | (df.population >= 25000000)][['name','population','area']]

In [49]:
big_countries(df)

,name,population,area
0,Afghanistan,25500100,652230
2,Algeria,37100000,2381741


# 181. Employees Earning More Than Their Managers

In [6]:
df = pd.read_clipboard()
df

,id,name,salary,managerId
0,1,Joe,70000,3.0
1,2,Henry,80000,4.0
2,3,Sam,60000,NaN
3,4,Max,90000,NaN


In [7]:
df = df[['id', 'name', 'salary', 'managerId']] # original df in lc
df['managerId'] = pd.to_numeric(df['managerId'], errors='coerce')
df

,id,name,salary,managerId
0,1,Joe,70000,3.0
1,2,Henry,80000,4.0
2,3,Sam,60000,NaN
3,4,Max,90000,NaN


In [88]:
def earn_more(df):
    tmp = df.merge(df, how='left', left_on='managerId', right_on='id')
    result = tmp[tmp.salary_x > tmp.salary_y][['name_x']].rename(columns={'name_x':'Employee'})
    return result

In [89]:
earn_more(df)

,Employee
0,Joe


# 175. Combine Two Tables

In [92]:
person = pd.read_clipboard()
person = person[['personId', 'lastName', 'firstName']]
person

,personId,lastName,firstName
0,1,Wang,Allen
1,2,Alice,Bob


In [99]:
address = pd.DataFrame(data=[[1,2,'New York City','New York'], [2,3,'Leetcode', 'California']], 
                       columns=['addressId', 'personId', 'city', 'state'])
address

,addressId,personId,city,state
0,1,2,New York City,New York
1,2,3,Leetcode,California


In [101]:
def combine2(person, address):
    tmp = person.merge(address, how='left', on='personId')
    return tmp[['firstName', 'lastName', 'city', 'state']]

combine2(person, address)

,firstName,lastName,city,state
0,Allen,Wang,NaN,NaN
1,Bob,Alice,New York City,New York


# 177. Nth Highest Salary

In [102]:
df1 = pd.DataFrame({'id':[1,2,3,4,5,6], 'salary': [100,50,300,400,450,450]}) # 如果有max salary不唯一，应该取第二大的salary
df2 = pd.DataFrame({'id':[1,2,3,4,5], 'salary': [100,50,300,400,450]})
df3 = pd.DataFrame({'id':[1], 'salary': [100]})
df4 = pd.DataFrame({'id':[], 'salary': []})

In [131]:
def find_NthHighestSalary(df, n):
    if len(df.salary.unique()) < n:
        return pd.DataFrame({'SecondHighestSalary':  [None]})
    df = df.copy()
    df['rnk'] = df.salary.rank(method='dense', ascending=False).astype(int)
    result = df[df.rnk==n]['salary'].unique()[0]
    return pd.DataFrame({'SecondHighestSalary':  [result]})

In [133]:
print(find_NthHighestSalary(df1, 5))
print(find_NthHighestSalary(df1, 2))
print(find_NthHighestSalary(df1, 1))
print(find_NthHighestSalary(df2, 4))
print(find_NthHighestSalary(df3, 1))
print(find_NthHighestSalary(df4, 5))

   SecondHighestSalary
0                   50
   SecondHighestSalary
0                  400
   SecondHighestSalary
0                  450
   SecondHighestSalary
0                  100
   SecondHighestSalary
0                  100
  SecondHighestSalary
0                None


# 184. Department Highest Salary

In [37]:
employee = pd.read_clipboard()
employee = employee[['id', 'name', 'salary', 'departmentId']]
employee

,id,name,salary,departmentId
0,1,Joe,70000,1
1,2,Jim,90000,1
2,3,Henry,80000,2
3,4,Sam,60000,2
4,5,Max,90000,1


In [38]:
department = pd.DataFrame(data=[[1, 'IT'], [2, 'Sales']], columns=['id', 'name'])
department 

,id,name
0,1,IT
1,2,Sales


In [51]:
def dept_highest_salary(employee, department):
    employee, department = employee.copy(), department.copy()
    
    employee['rnk'] = employee.groupby('departmentId')['salary'].rank(method='dense', ascending=False)
    
    employee = employee[employee.rnk == 1]
    result = employee.merge(department, how='left', left_on='departmentId', right_on='id')
    result = result[['name_y', 'name_x', 'salary']]
    result = result.rename(columns={"name_y":'department', 'name_x':'employee'})
    return result

dept_highest_salary(employee, department)

,department,employee,salary
0,IT,Jim,90000
1,Sales,Henry,80000
2,IT,Max,90000


# 185. Department Top Three Salaries

In [124]:
employee = pd.read_clipboard()
employee = employee[['id', 'name', 'salary', 'departmentId']]
employee

,id,name,salary,departmentId
0,1,Joe,85000,1
1,2,Henry,80000,2
2,3,Sam,60000,2
3,4,Max,90000,1
4,5,Janet,69000,1
5,6,Randy,85000,1
6,7,Will,70000,1


In [125]:
department = pd.DataFrame(data=[[1, 'IT'], [2, 'Sales']], columns=['id', 'name'])
department

,id,name
0,1,IT
1,2,Sales


In [136]:
e, d = employee.copy(), department.copy()
e['rnk'] = e.groupby('departmentId')['salary'].rank(method='dense', ascending=False)
e = e[e.rnk <= 3]
tmp = e.merge(d, how='left', left_on='departmentId', right_on='id')
tmp = tmp[['name_y', 'name_x', 'salary']]
tmp.rename(columns={'name_y': 'Department', 'name_x': 'Employee', 'salary':'Salary'}, inplace=True)
tmp.sort_values(by=['Department', 'Salary'], ascending=[True, False], inplace=True)
tmp

,Department,Employee,Salary
3,IT,Max,90000
0,IT,Joe,85000
4,IT,Randy,85000
5,IT,Will,70000
1,Sales,Henry,80000
2,Sales,Sam,60000


# 262. Trips and Users

记得再where里用IN/NOT IN;

In [176]:
trips = pd.read_clipboard()
trips = trips[['id', 'client_id', 'driver_id', 'city_id', 'status', 'request_at']]
trips

,id,client_id,driver_id,city_id,status,request_at
0,1,1,10,1,completed,2013-10-01
1,2,2,11,1,cancelled_by_driver,2013-10-01
2,3,3,12,6,completed,2013-10-01
3,4,4,13,6,cancelled_by_client,2013-10-01
4,5,1,10,1,completed,2013-10-02
5,6,2,11,6,completed,2013-10-02
6,7,3,12,6,completed,2013-10-02
7,8,2,12,12,completed,2013-10-03
8,9,3,10,12,completed,2013-10-03
9,10,4,13,12,cancelled_by_driver,2013-10-03


In [177]:
users = pd.read_clipboard()
users = users[['users_id', 'banned', 'role']]
users

,users_id,banned,role
0,1,No,client
1,2,Yes,client
2,3,No,client
3,4,No,client
4,10,No,driver
5,11,No,driver
6,12,No,driver
7,13,No,driver


### COUNT DISTINCT in Pandas:
SELECT count(distinct CLIENTCODE) FROM table GROUP BY YEARMONTH;

df.groupby('YEARMONTH').CLIENTCODE.nunique()


In [179]:
def find_cancel_rate(trips, users):
    not_banned = list(users[users.banned=='No']['users_id'])
    trips = trips[(trips.request_at>='2013-10-01') & (trips.request_at<='2013-10-03')]
    
    # Use the Series.isin() method to check if a series value is in a list of values
    trips = trips[(trips.client_id.isin(not_banned)) & (trips.driver_id.isin(not_banned))]
    
    trips['cancelled'] = trips['status'].apply(lambda x: 1 if x.startswith('cance') else 0)
    
    #trips.loc[trips['status'].str.startswith('cance', na=False), 'cancelled'] = 1
    #trips.loc[trips['status'].str.startswith('com', na=False), 'cancelled'] = 0
    
    result = pd.DataFrame(round(trips.groupby('request_at').sum('cancelled')['cancelled'] 
                                / trips.groupby('request_at').id.nunique(), 2))
    return result

find_cancel_rate(trips, users)

,0
request_at,
2013-10-01,0.33
2013-10-02,0.00
2013-10-03,0.50


# 1484. Group Sold Products By The Date

In [180]:
df = pd.read_clipboard()
df = df[['sell_date', 'product']]
df = df.dropna()
df

,sell_date,product
1,2020-05-30,Headphone
2,2020-06-01,Pencil
3,2020-06-02,Mask
4,2020-05-30,Basketball
5,2020-06-01,Bible
6,2020-06-02,Mask
7,2020-05-30,T-Shirt


In [204]:
tmp = df.drop_duplicates()
tmp1 = tmp.groupby('sell_date').agg({'product' : lambda x: ', '.join(x)})
tmp2 = tmp.groupby('sell_date').count().rename(columns={'product':'count'})
tmp2.join(tmp1)

,count,product
sell_date,,
2020-05-30,3,"Headphone, Basketball, T-Shirt"
2020-06-01,2,"Pencil, Bible"
2020-06-02,1,Mask


In [207]:
tmp.groupby('sell_date').agg({'product' : ['count', lambda x: ', '.join(x)]})

product                                
             count                      <lambda_0>
sell_date                                         
2020-05-30       3  Headphone, Basketball, T-Shirt
2020-06-01       2                   Pencil, Bible
2020-06-02       1                            Mask

# 1179. Reformat Department Table

In [281]:
df = pd.read_clipboard()
df = df[['id', 'revenue', 'month']]
df = df.dropna()
df

,id,revenue,month
1,1.0,8000.0,Jan
2,2.0,9000.0,Jan
3,3.0,10000.0,Feb
4,1.0,7000.0,Feb
5,1.0,6000.0,Mar


### use pd.pivot()

In [289]:
tmp = df.pivot(index='id', columns='month', values='revenue')
tmp = tmp[['Jan', 'Feb', 'Mar']]
tmp['Apr'] = np.nan
tmp['May'] = np.nan
tmp['Jun'] = np.nan
tmp['Jul'] = np.nan
tmp['Aug'] = np.nan
tmp['Sep'] = np.nan
tmp['Oct'] = np.nan
tmp['Nov'] = np.nan
tmp['Dec'] = np.nan
tmp

month,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
id,,,,,,,,,,,,
1.0,8000.0,7000.0,6000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,9000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### use sql logic

In [233]:
tmp = df.copy()
tmp['Jan_Revenue'] = df.apply(lambda row: row['revenue'] if row['month']=='Jan' else np.NaN, axis=1)
tmp['Feb_Revenue'] = df.apply(lambda row: row['revenue'] if row['month']=='Feb' else np.NaN, axis=1)
tmp['Mar_Revenue'] = df.apply(lambda row: row['revenue'] if row['month']=='Mar' else np.NaN, axis=1)
tmp['Apr_Revenue'] = df.apply(lambda row: row['revenue'] if row['month']=='Apr' else np.NaN, axis=1)
tmp['May_Revenue'] = df.apply(lambda row: row['revenue'] if row['month']=='May' else np.NaN, axis=1)
tmp['Jun_Revenue'] = df.apply(lambda row: row['revenue'] if row['month']=='Jun' else np.NaN, axis=1)
tmp['Jul_Revenue'] = df.apply(lambda row: row['revenue'] if row['month']=='Jul' else np.NaN, axis=1)
tmp['Aug_Revenue'] = df.apply(lambda row: row['revenue'] if row['month']=='Aug' else np.NaN, axis=1)
tmp['Sep_Revenue'] = df.apply(lambda row: row['revenue'] if row['month']=='Sep' else np.NaN, axis=1)
tmp['Oct_Revenue'] = df.apply(lambda row: row['revenue'] if row['month']=='Oct' else np.NaN, axis=1)
tmp['Nov_Revenue'] = df.apply(lambda row: row['revenue'] if row['month']=='Nov' else np.NaN, axis=1)
tmp['Dec_Revenue'] = df.apply(lambda row: row['revenue'] if row['month']=='Dec' else np.NaN, axis=1)
tmp

,id,revenue,month,Jan_Revenue,Feb_Revenue,Mar_Revenue,Apr_Revenue,May_Revenue,Jun_Revenue,Jul_Revenue,Aug_Revenue,Sep_Revenue,Oct_Revenue,Nov_Revenue,Dec_Revenue
1,1.0,8000.0,Jan,8000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,9000.0,Jan,9000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,10000.0,Feb,NaN,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,7000.0,Feb,NaN,7000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,6000.0,Mar,NaN,NaN,6000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [234]:
tmp = tmp.groupby('id').sum(['Jan_Revenue', 'Feb_Revenue', 'Mar_Revenue', 'Apr_Revenue', 'May_Revenue', 'Jun_Revenue', 'Jul_Revenue', 'Aug_Revenue', 'Sep_Revenue', 'Oct_Revenue', 'Nov_Revenue', 'Dec_Revenue'])\
         .reset_index()[['id', 'Jan_Revenue', 'Feb_Revenue', 'Mar_Revenue', 'Apr_Revenue', 'May_Revenue', 'Jun_Revenue', 'Jul_Revenue', 'Aug_Revenue', 'Sep_Revenue', 'Oct_Revenue', 'Nov_Revenue', 'Dec_Revenue']]
tmp

,id,Jan_Revenue,Feb_Revenue,Mar_Revenue,Apr_Revenue,May_Revenue,Jun_Revenue,Jul_Revenue,Aug_Revenue,Sep_Revenue,Oct_Revenue,Nov_Revenue,Dec_Revenue
0,1.0,8000.0,7000.0,6000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,9000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,0.0,10000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [235]:
tmp.replace(0, np.NaN, inplace=True)
tmp

,id,Jan_Revenue,Feb_Revenue,Mar_Revenue,Apr_Revenue,May_Revenue,Jun_Revenue,Jul_Revenue,Aug_Revenue,Sep_Revenue,Oct_Revenue,Nov_Revenue,Dec_Revenue
0,1.0,8000.0,7000.0,6000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,9000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,NaN,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 618. Students Report By Geography

In [326]:
df = pd.read_clipboard()
df = df[['name', 'continent']]
df = df.dropna()
df

,name,continent
1,Jane,America
2,Pascal,Europe
3,Xi,Asia
4,Jack,America


In [330]:
tmp = df.copy()
tmp['row_number'] = tmp.groupby('continent').rank(method='first')
display(tmp)
tmp.pivot(index='row_number', columns='continent', values='name')

,name,continent,row_number
1,Jane,America,2.0
2,Pascal,Europe,1.0
3,Xi,Asia,1.0
4,Jack,America,1.0


continent,America,Asia,Europe
row_number,,,
1.0,Jack,Xi,Pascal
2.0,Jane,NaN,NaN


# 1972. First and Last Call On the Same Day

In [255]:
df = pd.read_clipboard()
#df = df[['caller_id', 'recipient_id', 'call_time']]
#df = df.dropna()
df

,caller_id,recipient_id,call_time
0,8,4,2021-08-24 17:46:07
1,4,8,2021-08-24 19:57:13
2,5,1,2021-08-11 05:28:44
3,8,3,2021-08-17 04:04:15
4,11,3,2021-08-17 13:07:00
5,8,11,2021-08-17 22:22:22


In [276]:
u = pd.concat([df[['caller_id', 'recipient_id', 'call_time']].rename(columns={'caller_id':'user1', 'recipient_id': 'user2'}), 
               df[['recipient_id', 'caller_id', 'call_time']].rename(columns={'recipient_id':'user1', 'caller_id': 'user2'})])
u['call_time'] = pd.to_datetime(u['call_time'])
u['call_date'] = u['call_time'].dt.date
u['rnk1'] = u.groupby(['user1', 'call_date'])['call_time'].rank(method='dense', ascending=True)
u['rnk2'] = u.groupby(['user1', 'call_date'])['call_time'].rank(method='dense', ascending=False)
u

,user1,user2,call_time,call_date,rnk1,rnk2
0,8,4,2021-08-24 17:46:07,2021-08-24,1.0,2.0
1,4,8,2021-08-24 19:57:13,2021-08-24,2.0,1.0
2,5,1,2021-08-11 05:28:44,2021-08-11,1.0,1.0
3,8,3,2021-08-17 04:04:15,2021-08-17,1.0,2.0
4,11,3,2021-08-17 13:07:00,2021-08-17,1.0,2.0
5,8,11,2021-08-17 22:22:22,2021-08-17,2.0,1.0
0,4,8,2021-08-24 17:46:07,2021-08-24,1.0,2.0
1,8,4,2021-08-24 19:57:13,2021-08-24,2.0,1.0
2,1,5,2021-08-11 05:28:44,2021-08-11,1.0,1.0
3,3,8,2021-08-17 04:04:15,2021-08-17,1.0,2.0


In [280]:
tmp = u.merge(u, how='inner', on=['user1', 'user2', 'call_date'])
tmp[(tmp['rnk1_x']==1) & (tmp['rnk2_y']==1)]['user1'].unique()

array([8, 4, 5, 1])

# 180. Consecutive Numbers

In [11]:
df = pd.DataFrame({'id':[1,2,3,4,5,6,7,8,9,10], 'num':[1,1,1,2,1,2,2,5,5,5]})
df

,id,num
0,1,1
1,2,1
2,3,1
3,4,2
4,5,1
5,6,2
6,7,2
7,8,5
8,9,5
9,10,5


applying leads and lags by shifting dataframe column up and down respectively

In [245]:
def consec_nums(df):
    df['pre'] = df.num.shift(1)
    df['post'] = df.num.shift(-1)
    unique_cons_nums = df[(df.num==df.post) & (df.num==df.pre) & (df.pre==df.post)]['num'].unique()
    # return df[(df.num == df.post) & (df.post == df.pre)][['num']].drop_duplicates().rename(columns = {'num': 'ConsecutiveNums'})
    return pd.DataFrame({'ConsecutiveNums':unique_cons_nums})

consec_nums(df)

,ConsecutiveNums
0,1
1,5


# 196. Delete Duplicate Emails

In [52]:
df = pd.DataFrame({'id':[1,2,3], 'email':['john@example.com', 'Bob@example.com', 'john@example.com']})
df

,id,email
0,1,john@example.com
1,2,Bob@example.com
2,3,john@example.com


`not in` operation use `~`

In [254]:
df[~df.id.isin(list(df.groupby(['email']).min('id')['id']))]

,id,email
2,3,john@example.com


In [256]:
def delete_dup(df):
    df = df[df.id.isin(list(df.groupby(['email']).min('id')['id']))]
    return df

delete_dup(df)

,id,email
0,1,john@example.com
1,2,Bob@example.com


# 608. Tree Node

In [104]:
df = pd.read_clipboard()
df = df[['id', 'p_id']]
df

,id,p_id
1,1.0,NaN
2,2.0,1.0
3,3.0,1.0
4,4.0,2.0
5,5.0,2.0


In [123]:
import math
tmp = df.copy()
def func(x):
    if math.isnan(x['p_id']):
        return 'root'
    elif x['id'] in list(tmp.p_id):
        return 'inner'
    return 'leaf'
tmp['type'] = tmp.apply(func, axis=1)
tmp[['id', 'type']]

,id,type
1,1.0,root
2,2.0,inner
3,3.0,leaf
4,4.0,leaf
5,5.0,leaf


# 626. Exchange Seats

In [97]:
df = pd.read_clipboard()
df = df[['id', 'student']]
df = df.dropna()
df

,id,student
1,1.0,Abbot
2,2.0,Doris
3,3.0,Emerson
4,4.0,Green
5,5.0,Jeames


In [103]:
tmp = df.copy()
tmp['nxt'] = tmp.student.shift(-1)
tmp['pre'] = tmp.student.shift(1)
tmp['result'] = tmp.apply(lambda x: x['nxt'] if x['id']%2==1 else x['pre'], axis=1)
# tmp['result'] = np.where(tmp['result'].isna(), tmp['student'], tmp['result'])
tmp.loc[tmp['result'].isna(), 'result'] = tmp['student']
tmp[['id', 'result']]

,id,result
1,1.0,Doris
2,2.0,Abbot
3,3.0,Green
4,4.0,Emerson
5,5.0,Jeames


# 183. Customers Who Never Order

In [24]:
Customers = pd.DataFrame({'id':[1,2,3,4], 'name':['Joe', 'Henry', 'Sam', 'Max']})
Customers

,id,name
0,1,Joe
1,2,Henry
2,3,Sam
3,4,Max


In [25]:
Orders = pd.DataFrame({'id':[1,2], 'customerId':[3,1]})
Orders

,id,customerId
0,1,3
1,2,1


In [265]:
def never_order(Customers, Orders):
    ordered_customers = Orders.customerId.unique()
    return Customers[~Customers.id.isin(ordered_customers)][['name']]

never_order(Customers, Orders)

,name
1,Henry
3,Max


# 511. Game Play Analysis I

In [29]:
df = pd.read_clipboard()
df = df[['player_id', 'device_id', 'event_date', 'games_played']]
df

,player_id,device_id,event_date,games_played
0,1,2,2016-03-01,5
1,1,2,2016-05-02,6
2,2,3,2017-06-25,1
3,3,1,2016-03-02,0
4,3,4,2018-07-03,5


In [274]:
def first_login_date(df):
    return df.groupby('player_id').min()[['event_date']]

first_login_date(df)

,event_date
player_id,
1,2016-03-01
2,2017-06-25
3,2016-03-02


# 178. Rank Scores

In [275]:
df = pd.DataFrame({'id':[1,2,3,4,5,6], 'score':[3.50,3.65,4.00,3.85,4.00,3.65]})
df

,id,score
0,1,3.50
1,2,3.65
2,3,4.00
3,4,3.85
4,5,4.00
5,6,3.65


In [278]:
def rank_scores(df):
    df['rank'] = df.score.rank(method='dense', ascending=False)
    df = df.sort_values('rank')[['score', 'rank']]
    return df
rank_scores(df)

,score,rank
2,4.00,1.0
4,4.00,1.0
3,3.85,2.0
1,3.65,3.0
5,3.65,3.0
0,3.50,4.0


# 2004. The Number of Seniors and Juniors to Join the Company

`df.salary.cumsum()` == `sum(salary) over (rows between unbounded preceding and current row)`

In [279]:
df = pd.read_clipboard()
df = df[['employee_id', 'experience', 'salary']]
df

,employee_id,experience,salary
0,1,Junior,10000
1,9,Junior,10000
2,2,Senior,20000
3,11,Senior,20000
4,13,Senior,50000
5,4,Junior,40000


In [303]:
def seniors_juniors(df):
    senior = df[df.experience=='Senior'].sort_values('salary')
    junior = df[df.experience=='Junior'].sort_values('salary')
    senior['cumulative_salary'] = senior.salary.cumsum()
    num_senior = len(senior[senior.cumulative_salary <= 70000])
    cumulative_senior_salary = max(senior[senior.cumulative_salary <= 70000]['cumulative_salary'])
    
    junior['cumulative_salary'] = cumulative_senior_salary + junior.salary.cumsum()
    num_junior = len(junior[junior.cumulative_salary <= 70000])
    
    return pd.DataFrame({'experience':['Senior', 'Junior'], 'accepted_candidates': [num_senior, num_junior]})

seniors_juniors(df)

,experience,accepted_candidates
0,Senior,2
1,Junior,2


# 197. Rising Temperature

In [310]:
df = pd.read_clipboard()
df = df[['id', 'recordDate', 'temperature']]
df["recordDate"] = pd.to_datetime(df["recordDate"])
df

,id,recordDate,temperature
0,1,2015-01-01,10
1,2,2015-01-02,25
2,3,2015-01-03,20
3,4,2015-01-04,30


In [320]:
def rising_temp(df):
    tmp = df.copy()
    tmp['yesterday'] = tmp["recordDate"] - timedelta(days=1)
    tmp = tmp.merge(tmp, how='left', left_on='yesterday', right_on='recordDate')
    result = tmp[tmp.temperature_x > tmp.temperature_y][['id_x']]
    result.column = ['id']
    return result

rising_temp(df)

/Users/yujiaxie/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


,id_x
1,2
3,4


# 182. Duplicate Emails

In [321]:
df = pd.DataFrame({'id':[1,2,3], 'email':['a@b.com', 'c@d.com', 'a@b.com']})
df

,id,email
0,1,a@b.com
1,2,c@d.com
2,3,a@b.com


In [328]:
def get_dup(df):
    tmp = df.groupby('email').count().reset_index()
    return tmp[tmp.id>=2][['email']]

get_dup(df)

,email
0,a@b.com


# 1795. Rearrange Products Table

unpivot: columns to rows: use union/ union all

In [143]:
df = pd.DataFrame({'product_id':[0,1], 'store1':[95,70], 'store2':[100, np.nan], 'store3':[105,80]})
df

,product_id,store1,store2,store3
0,0,95,100.0,105
1,1,70,NaN,80


use `df.melt()` for columns to rows 

In [146]:
result = df.melt(id_vars=['product_id'], value_vars=['store1', 'store2', 'store3'],
        var_name="store", 
        value_name="price")
result[~result.price.isna()].sort_values(['product_id', 'store'])

,product_id,store,price
0,0,store1,95.0
2,0,store2,100.0
4,0,store3,105.0
1,1,store1,70.0
5,1,store3,80.0


# 627. Swap Salary

In [147]:
df = pd.read_clipboard()
df = df[['id', 'name', 'sex', 'salary']]
df = df.dropna()
df

,id,name,sex,salary
1,1.0,A,m,2500.0
2,2.0,B,f,1500.0
3,3.0,C,m,5500.0
4,4.0,D,f,500.0


In [151]:
tmp = df.copy()
tmp['swapped'] = tmp.sex.apply(lambda x: 'm' if x=='f' else 'f')
tmp[['id','name', 'swapped', 'salary']].rename(columns={'swapped':'sex'})

,id,name,sex,salary
1,1.0,A,f,2500.0
2,2.0,B,m,1500.0
3,3.0,C,f,5500.0
4,4.0,D,m,500.0


# 1141. User Activity for the Past 30 Days I

In [152]:
df = pd.read_clipboard()
df = df[['user_id', 'session_id', 'activity_date', 'activity_type']]
df = df.dropna()
df

,user_id,session_id,activity_date,activity_type
1,1.0,1.0,2019-07-20,open_session
2,1.0,1.0,2019-07-20,scroll_down
3,1.0,1.0,2019-07-20,end_session
4,2.0,4.0,2019-07-20,open_session
5,2.0,4.0,2019-07-21,send_message
6,2.0,4.0,2019-07-21,end_session
7,3.0,2.0,2019-07-21,open_session
8,3.0,2.0,2019-07-21,send_message
9,3.0,2.0,2019-07-21,end_session
10,4.0,3.0,2019-06-25,open_session


In [175]:
tmp = df[['user_id', 'activity_date']].drop_duplicates()
tmp['activity_date'] = pd.to_datetime(tmp.activity_date)
tmp = tmp[(tmp.activity_date < '2019-07-27') & (tmp.activity_date > pd.to_datetime('2019-07-27')-timedelta(days=30))]
tmp.groupby('activity_date').count()[['user_id']].rename(columns={'user_id':'active_users'}).reset_index()

,activity_date,active_users
0,2019-07-20,2
1,2019-07-21,2


# 1841. League Statistics

In [341]:
teams = pd.DataFrame({'team_id':[1,4,6], 'team_name':['Ajax', 'Dortmund', 'Arsenal']})
teams

,team_id,team_name
0,1,Ajax
1,4,Dortmund
2,6,Arsenal


In [340]:
matches = pd.DataFrame({'home_team_id':[1,1,4,6], 'away_team_id':[4,6,1,1], 'home_team_goals':[0,3,5,0], 'away_team_goals':[1,3,2,0]})
matches

,home_team_id,away_team_id,home_team_goals,away_team_goals
0,1,4,0,1
1,1,6,3,3
2,4,1,5,2
3,6,1,0,0


- Use `.apply` with `axis=1` to send every single row to a function
- `df[df.columns.difference(['b'])]` to select all columns except `b`

In [351]:
def home_cal_point(row):
    if row['home_team_goals'] > row['away_team_goals']:
        return 3
    elif row['home_team_goals'] == row['away_team_goals']:
        return 1
    else:
        return 0
    
def away_cal_point(row):
    if row['home_team_goals'] < row['away_team_goals']:
        return 3
    elif row['home_team_goals'] == row['away_team_goals']:
        return 1
    else:
        return 0

In [364]:
m, t = matches.copy(), teams.copy()
m['home_team_point'] = m.apply(home_cal_point, axis=1)
m['away_team_point'] = m.apply(away_cal_point, axis=1)
m = pd.merge(m, t, how='left', left_on='home_team_id', right_on='team_id')
m = m[m.columns.difference(['team_id'])]
m = pd.merge(m, t, how='left', left_on='away_team_id', right_on='team_id')
m = m[m.columns.difference(['team_id'])]
m.rename({'team_name_x': 'home_team_name'}, axis=1, inplace=True)
m.rename({'team_name_y': 'away_team_name'}, axis=1, inplace=True)
m

,away_team_goals,away_team_id,away_team_point,home_team_goals,home_team_id,home_team_point,home_team_name,away_team_name
0,1,4,3,0,1,0,Ajax,Dortmund
1,3,6,1,3,1,1,Ajax,Arsenal
2,2,1,0,5,4,3,Dortmund,Ajax
3,0,1,1,0,6,1,Arsenal,Ajax


In [391]:
gb_home = m.groupby(['home_team_name'])['home_team_name', 'home_team_point', 'home_team_goals', 'away_team_goals']\
        .agg(['sum', 'count'])
gb_away = m.groupby(['away_team_name'])['away_team_name', 'away_team_point', 'away_team_goals', 'home_team_goals']\
        .agg(['sum', 'count'])

/Users/yujiaxie/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.
/Users/yujiaxie/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [400]:
home = pd.DataFrame({#'home_team_name':list(gb_home.index), 
                     'home_matches_played':gb_home['home_team_name']['count'],
                     'home_points':gb_home['home_team_point']['sum'],
                     'home_goals_for':gb_home['home_team_goals']['sum'], 
                     'home_goals_against':gb_home['away_team_goals']['sum']}).reset_index()
away = pd.DataFrame({#'away_team_name':list(gb_away.index), 
                     'away_matches_played':gb_away['away_team_name']['count'],
                     'away_points':gb_away['away_team_point']['sum'],
                     'away_goals_for':gb_away['away_team_goals']['sum'], 
                     'away_goals_against':gb_away['home_team_goals']['sum']}).reset_index()

In [401]:
home

,home_matches_played,home_points,home_goals_for,home_goals_against
home_team_name,,,,
Ajax,2,1,3,4
Arsenal,1,1,0,0
Dortmund,1,3,5,2


In [411]:
result = home.join(away, how='outer').reset_index()
result['matches_played'] = result.home_matches_played+result.away_matches_played
result['points'] = result.home_points+result.away_points
result['goals_for'] = result.home_goals_for+result.away_goals_for
result['goals_against'] = result.home_goals_against+result.away_goals_against
result['goals_diff'] = result.goals_for-result.goals_against
result = result[['home_team_name', 'matches_played', 'points', 'goals_for', 'goals_against', 'goals_diff']]\
         .rename({'home_team_name':'team_name'}, axis=1)
result.sort_values(by=['points','goals_diff', 'team_name'], inplace=True,ascending = [False,False, True])
result

,team_name,matches_played,points,goals_for,goals_against,goals_diff
2,Dortmund,2,6,6,2,4
1,Arsenal,2,2,3,3,0
0,Ajax,4,2,5,9,-4


# 1454. Active Users

In [413]:
accounts = pd.DataFrame({'id':[1,7], 'name':['Winston', 'Jonathan']})
accounts

,id,name
0,1,Winston
1,7,Jonathan


In [438]:
logins = pd.read_clipboard()
logins = logins[['id', 'login_date']]
logins['login_date'] = pd.to_datetime(logins['login_date'])
logins

,id,login_date
0,7,2020-05-30
1,1,2020-05-30
2,7,2020-05-31
3,7,2020-06-01
4,7,2020-06-02
5,7,2020-06-02
6,7,2020-06-03
7,1,2020-06-07
8,7,2020-06-10


The `method='first'` for the `rank()` method for pandas series is equivalent to the `ROW_NUMBER()` window function in SQL.

Or use `sort_values()`, `groupby()` and finally `cumcount() + 1`:

In [439]:
logins.drop_duplicates(inplace=True)
logins = logins.sort_values(['id', 'login_date'])
logins.reset_index(drop=True, inplace=True)
logins

,id,login_date
0,1,2020-05-30
1,1,2020-06-07
2,7,2020-05-30
3,7,2020-05-31
4,7,2020-06-01
5,7,2020-06-02
6,7,2020-06-03
7,7,2020-06-10


In [477]:
logins['rnk'] = logins.groupby('id').cumcount() + 1
# logins['rnk'] = logins.groupby('id').rank(method='first')
logins['date_diff'] = logins['login_date'] - pd.to_timedelta(logins['rnk'], unit='d')
tmp = logins.groupby(['id', 'date_diff']).count()
tmp = tmp[tmp['login_date']>=5].reset_index()  
tmp.merge(accounts, on='id')[['id', 'name']]

,id,name
0,7,Jonathan


# 601. Human Traffic of Stadium

In [291]:
df = pd.read_clipboard()
df = df[['id', 'visit_date', 'people']]
df = df.dropna()
df

,id,visit_date,people
1,1.0,2017-01-01,10.0
2,2.0,2017-01-02,109.0
3,3.0,2017-01-03,150.0
4,4.0,2017-01-04,99.0
5,5.0,2017-01-05,145.0
6,6.0,2017-01-06,1455.0
7,7.0,2017-01-07,199.0
8,8.0,2017-01-09,188.0


### xx.shift(-1) == lead(xx, 1) over ()
### xx.shift(1) == lag(xx, 1) over ()

In [302]:
tmp = df.copy()
tmp['id1'] = tmp.id.shift(-1)
tmp['id2'] = tmp.id.shift(-2)
tmp['people1'] = tmp.people.shift(-1)
tmp['people2'] = tmp.people.shift(-2)
tmp = tmp[(tmp['people']>=100) & (tmp['people1']>=100) & (tmp['people2']>=100)]
ids = set(list(tmp.id) + list(tmp.id1) + list(tmp.id2))
df[df.id.isin(ids)]

,id,visit_date,people
5,5.0,2017-01-05,145.0
6,6.0,2017-01-06,1455.0
7,7.0,2017-01-07,199.0
8,8.0,2017-01-09,188.0


# 550. Game Play Analysis IV

In [303]:
df = pd.read_clipboard()
df = df[['player_id', 'device_id', 'event_date', 'games_played']]
df = df.dropna()
df

,player_id,device_id,event_date,games_played
1,1.0,2.0,2016-03-01,5.0
2,1.0,2.0,2016-03-02,6.0
3,2.0,3.0,2017-06-25,1.0
4,3.0,1.0,2016-03-02,0.0
5,3.0,4.0,2018-07-03,5.0


### .groupby('player_id')['event_date'].transform('min') <==> min(event_date) over (partition by player_id)

In [312]:
tmp = df.copy()
tmp['event_date'] = pd.to_datetime(tmp['event_date'])
tmp['first_logged_in'] = tmp.groupby('player_id')['event_date'].transform('min')
numerator = len(tmp[tmp['event_date'] == tmp['first_logged_in'] + timedelta(days=1)])
numerator / len(tmp.player_id.unique())

0.3333333333333333

# 1194. Tournament Winners

In [315]:
players = pd.read_clipboard()
players = players[['player_id', 'group_id']]
players = players.dropna()
players

,player_id,group_id
1,15.0,1.0
2,25.0,1.0
3,30.0,1.0
4,45.0,1.0
5,10.0,2.0
6,35.0,2.0
7,50.0,2.0
8,20.0,3.0
9,40.0,3.0


In [316]:
matches = pd.read_clipboard()
matches = matches[['match_id', 'first_player', 'second_player', 'first_score', 'second_score']]
matches = matches.dropna()
matches

,match_id,first_player,second_player,first_score,second_score
1,1.0,15.0,45.0,3.0,0.0
2,2.0,30.0,25.0,1.0,2.0
3,3.0,30.0,15.0,2.0,0.0
4,4.0,40.0,20.0,5.0,2.0
5,5.0,35.0,50.0,1.0,1.0


In [325]:
u = pd.concat([matches[['match_id', 'first_player', 'first_score']].rename(columns={'first_player': 'player_id', 'first_score':'score'}), 
               matches[['match_id', 'second_player', 'second_score']].rename(columns={'second_player': 'player_id', 'second_score':'score'})])
tmp = pd.DataFrame(u.groupby('player_id')['score'].sum()).reset_index()
tmp = tmp.merge(players, how='inner', on='player_id')
tmp['row_number'] = tmp.groupby('group_id')['score'].rank(method='first', ascending=False)
tmp[tmp['row_number'] == 1][['group_id', 'player_id']]


,group_id,player_id
0,1.0,15.0
4,2.0,35.0
5,3.0,40.0


# 2474. Customers With Strictly Increasing Purchases

In [331]:
df = pd.read_clipboard()
df = df[['order_id', 'customer_id', 'order_date', 'price']]
df = df.dropna()
df

,order_id,customer_id,order_date,price
1,1.0,1.0,2019-07-01,1100.0
2,2.0,1.0,2019-11-01,1200.0
3,3.0,1.0,2020-05-26,3000.0
4,4.0,1.0,2021-08-31,3100.0
5,5.0,1.0,2022-12-07,4700.0
6,6.0,2.0,2015-01-01,700.0
7,7.0,2.0,2017-11-07,1000.0
8,8.0,3.0,2017-01-01,900.0
9,9.0,3.0,2018-11-07,900.0


In [345]:
df['order_date'] = pd.to_datetime(df['order_date'])
tmp = df.copy()
tmp['order_year'] = tmp['order_date'].dt.year
tmp = tmp.groupby(['customer_id','order_year']).sum()[['price']].reset_index()
tmp['last_year'] = tmp.order_year-1

tmp = tmp.merge(tmp, how='left', left_on=['customer_id', 'last_year'], right_on=['customer_id', 'order_year'])
tmp['indicator'] = tmp['price_x'] > tmp['price_y']
tmp['s'] = tmp.groupby('customer_id')['indicator'].transform('sum') 
tmp['c'] = tmp.groupby('customer_id')['indicator'].transform('count') 
tmp[(tmp.s == tmp.c-1) | (tmp.c==1)]['customer_id'].unique()

array([1.])

# 1285. Find the Start and End Number of Continuous Ranges

In [346]:
df = pd.read_clipboard()
df = df[['log_id']]
df = df.dropna()
df

,log_id
1,1.0
2,2.0
3,3.0
4,7.0
5,8.0
6,10.0


In [349]:
tmp = df.copy()
tmp['row_number'] = df.rank(method='first')
tmp['diff'] = tmp['log_id'] - tmp['row_number']
tmp.groupby('diff').agg({'log_id':['min', 'max']})

log_id      
        min   max
diff             
0.0     1.0   3.0
3.0     7.0   8.0
4.0    10.0  10.0

# 1398. Customers Who Bought Products A and B but Not C

In [352]:
customers = pd.read_clipboard()
customers = customers[['customer_id', 'customer_name']]
customers = customers.dropna()
customers

,customer_id,customer_name
1,1.0,Daniel
2,2.0,Diana
3,3.0,Elizabeth
4,4.0,Jhon


In [351]:
orders = pd.read_clipboard()
orders = orders[['order_id', 'customer_id', 'product_name']]
orders = orders.dropna()
orders

,order_id,customer_id,product_name
1,10.0,1.0,A
2,20.0,1.0,B
3,30.0,1.0,D
4,40.0,1.0,C
5,50.0,2.0,A
6,60.0,3.0,A
7,70.0,3.0,B
8,80.0,3.0,D
9,90.0,4.0,C


In [361]:
o, c = orders.copy(), customers.copy()
tmp = o.groupby('customer_id').agg({'product_name': lambda x: any(x=='A') and any(x=='B') and not any(x=='C')}).reset_index()
#display(tmp)
tmp = tmp[tmp.product_name == True][['customer_id']]
tmp.merge(c, how='left', on='customer_id')

,customer_id,customer_name
0,3.0,Elizabeth


# 585. Investments in 2016

In [366]:
df = pd.read_clipboard()
df = df[['pid', 'tiv_2015', 'tiv_2016', 'lat', 'lon']]
df = df.dropna()
df

,pid,tiv_2015,tiv_2016,lat,lon
1,1.0,10.0,5.0,10.0,10.0
2,2.0,20.0,20.0,20.0,20.0
3,3.0,10.0,30.0,20.0,20.0
4,4.0,10.0,40.0,40.0,40.0


In [375]:
tmp = df.copy()
tmp['cnt'] = tmp.groupby('tiv_2015')['pid'].transform('count')
tmp['unique_loc'] = tmp.groupby(['lat', 'lon'])['pid'].transform('count')
round(tmp[(tmp['cnt']>1) & (tmp['unique_loc']==1)]['tiv_2016'].sum(), 2)

45.0

# 1097. Game Play Analysis V

In [376]:
df = pd.read_clipboard()
df = df[['player_id', 'device_id', 'event_date', 'games_played']]
df = df.dropna()
df['event_date'] = pd.to_datetime(df['event_date'])
df

,player_id,device_id,event_date,games_played
1,1.0,2.0,2016-03-01,5.0
2,1.0,2.0,2016-03-02,6.0
3,2.0,3.0,2017-06-25,1.0
4,3.0,1.0,2016-03-01,0.0
5,3.0,4.0,2016-07-03,5.0


In [379]:
t1 = df[['player_id', 'event_date']].drop_duplicates()
t1['rnk'] = t1.groupby('player_id')['event_date'].rank(method='first')
t1

,player_id,event_date,rnk
1,1.0,2016-03-01,1.0
2,1.0,2016-03-02,2.0
3,2.0,2017-06-25,1.0
4,3.0,2016-03-01,1.0
5,3.0,2016-07-03,2.0


In [385]:
t2 = t1[t1.rnk==1].groupby('event_date')[['player_id']].count().reset_index()
t2.columns = ['install_dt', 'installs']
t2

,install_dt,installs
0,2016-03-01,2
1,2017-06-25,1


In [400]:
t1['date2'] = t1['event_date'] + timedelta(days=1)
t3 = t1[t1['rnk']==1].merge(t1, how='inner', left_on=['date2', 'player_id'], right_on=['event_date', 'player_id'])
t3

,player_id,event_date_x,rnk_x,date2_x,event_date_y,rnk_y,date2_y
0,1.0,2016-03-01,1.0,2016-03-02,2016-03-02,2.0,2016-03-03


In [406]:
reselt = t2.merge(t3.groupby('event_date_x').count()[['player_id']], how='left', left_on='install_dt', right_on='event_date_x')
reselt = reselt.fillna(0).rename(columns={'player_id': "Day1_retention"})
reselt['Day1_retention'] = round(reselt['Day1_retention']/reselt.installs, 2)
reselt


,install_dt,installs,Day1_retention
0,2016-03-01,2,0.5
1,2017-06-25,1,0.0


# 1084. Sales Analysis III

In [407]:
Product = pd.read_clipboard()
Product = Product[['product_id', 'product_name', 'unit_price']]
Product = Product.dropna()
Product

,product_id,product_name,unit_price
1,1.0,S8,1000.0
2,2.0,G4,800.0
3,3.0,iPhone,1400.0


In [408]:
Sales = pd.read_clipboard()
Sales = Sales[['seller_id', 'product_id', 'buyer_id', 'sale_date', 'quantity', 'price']]
Sales = Sales.dropna()
Sales

,seller_id,product_id,buyer_id,sale_date,quantity,price
1,1.0,1.0,1.0,2019-01-21,2.0,2000.0
2,1.0,2.0,2.0,2019-02-17,1.0,800.0
3,2.0,2.0,3.0,2019-06-02,1.0,800.0
4,3.0,3.0,4.0,2019-05-13,2.0,2800.0


In [424]:
p, s = Product.copy(), Sales.copy()
s['indicator'] = s['sale_date'].apply(lambda x: x >='2019-01-01' and x <='2019-03-31')
res = s.groupby('product_id').agg({'product_id':'count', 'indicator':'sum'}).rename(columns={'product_id':'cnt', 'indicator':'s'}).reset_index()
p[p.product_id.isin(res[res.cnt == res.s]['product_id'])][['product_id', 'product_name']]


,product_id,product_name
1,1.0,S8


In [ ]:
df = pd.read_clipboard()
df = df[['order_id', 'customer_id', 'order_date', 'price']]
df = df.dropna()
df

In [ ]:
df = pd.read_clipboard()
df = df[['order_id', 'customer_id', 'order_date', 'price']]
df = df.dropna()
df

In [ ]:
df = pd.read_clipboard()
df = df[['order_id', 'customer_id', 'order_date', 'price']]
df = df.dropna()
df